In [33]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from tqdm import tqdm

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
#Reading data set using pandas
data = pd.read_csv("/content/drive/MyDrive/Reviews.csv")

In [36]:
sorted_data=data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [37]:
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)

In [39]:
data_negative_reviews = final[final.Score<3]  
data_positive_reviews = final[final.Score>3]  

In [40]:
data_negative = data_negative_reviews [0:1000]
data_positive = data_positive_reviews [0:1000]

In [41]:
data_negative = data_negative["Text"]
data_positive = data_positive["Text"]

In [42]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [43]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [44]:
from bs4 import BeautifulSoup
from tqdm import tqdm
def cleaning (data):
    preprocessed_reviews = []
    # tqdm is for printing the status bar
    for sentance in tqdm(data.values):
        # remove urls from text python: https://stackoverflow.com/a/40823105/4084039
        sentance = re.sub(r"http\S+", "", sentance)
        # https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
        sentance = BeautifulSoup(sentance, 'lxml').get_text()
        sentance = decontracted(sentance)
        #remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
        sentance = re.sub("\S*\d\S*", "", sentance).strip()
        #remove spacial character: https://stackoverflow.com/a/5843547/4084039
        sentance = re.sub('[^A-Za-z]+', ' ', sentance)
        # https://gist.github.com/sebleier/554280
        sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
        preprocessed_reviews.append(sentance.strip())
    lines = []

    for i in preprocessed_reviews:
        lines.append(i)

    data = ""

    for i in preprocessed_reviews:
        data = ' '. join(lines)
    z = []

    for i in data.split():
        z.append(i)

    data = ' '.join(z)
    return data

In [45]:
data_p = cleaning(data_positive)

100%|██████████| 1000/1000 [00:00<00:00, 2019.48it/s]


In [46]:
data_n = cleaning(data_negative)

100%|██████████| 1000/1000 [00:00<00:00, 2010.12it/s]


In [50]:
data_p[4001:]

' soup melody not found jingle accompany book get audio version immediately not sooner sing whoopy whoopy twice whoopy chicken soup rice correct oomph missing take chicken soup rice readings seriously book chicka chicka boom boom day still remains catchiest method teach kids months year also seriously danger forgotten pull old accordion strap dancing shoes time yukkin merry dance poultry broth sendak finest great little book read aloud nice rhythm well good repetition little ones like lines chicken soup rice child gets go months year go wonderful places like bombay nile eating well know get eat kids maurice sendak version ice skating treat roses heads long time not even know came surprise came little witty book book poetry months year goes month cute little poem go along love book really fun way learn months poems creative author purpose writing book give children fun way learn months children also learn things poetry rhythm reading book favorite book mine little girl would read time m

In [51]:
data_n[40003:]

'like pretty good deal ice cream sandwiches one review talked good deal agreed tried found paid one single ice cream sandwich process filing complaints trying get fixed considering another person posted problem suspecting something scam not scam poor quality control shipping single package got looked good foil wrapped unfortunately not eat dispute resolved oh took nearly weeks get one packages sent via extremely thin packaging left ice cream smashed bits postal service see resolution goes not expect order seller returned due short brief expiration date sept would less use pods amazon pull shelves stay away ordering company received order time amazment coffee pods stamped date months past due based upon various reviews decided give try normally use starbucks espresso pods say taste espressione much bland coffee tastes weaker shipment pods came best date full months delivery aware amazon lists grocery items non returnable really looking forward pods based reviews starbucks good prefer bo

In [81]:
#positive words test data
data_p_test = data_p[4001:4200]
data_p_test

' soup melody not found jingle accompany book get audio version immediately not sooner sing whoopy whoopy twice whoopy chicken soup rice correct oomph missing take chicken soup rice readings seriously'

In [82]:
#Negative words test data
data_n_test = data_n[4003:4200]
data_n_test

' still nothing get stuck actually saw second fly land watched flapped wings frantically within secs unstuck product total waste money could rate fly trap lower one star would think flies come miles'

In [56]:
#function to predict next word
def predict_Next_word(model,tokenizer,text):
  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  return predicted_word

In [57]:
#Loading Model and tokenizer
from tensorflow.keras.models import load_model
import numpy as np
import pickle
def Predict_Next_Words_LSTM_P(text):
  model = load_model('/content/drive/MyDrive/Deep_Learning/next_words_p.h5')
  tokenizer = pickle.load(open('/content/drive/MyDrive/Deep_Learning/tokenizer_p.pkl', 'rb'))
  temp = predict_Next_word(model,tokenizer,text)
  return temp
def Predict_Next_Words_LSTM_N(text):
  model = load_model('/content/drive/MyDrive/Deep_Learning/next_words_n.h5')
  tokenizer = pickle.load(open('/content/drive/MyDrive/Deep_Learning/tokenizer_n.pkl', 'rb'))
  temp = predict_Next_word(model,tokenizer,text)
  return temp
def Predict_Next_Words_BILSTM_P(text):
  model = load_model('/content/drive/MyDrive/Deep_Learning/next_words_p_bi.h5')
  tokenizer = pickle.load(open('/content/drive/MyDrive/Deep_Learning/tokenizer_p.pkl', 'rb'))
  temp = predict_Next_word(model,tokenizer,text)
  return temp
def Predict_Next_Words_BILSTM_N(text):
  model = load_model('/content/drive/MyDrive/Deep_Learning/next_words_n_bi.h5')
  tokenizer = pickle.load(open('/content/drive/MyDrive/Deep_Learning/tokenizer_n.pkl', 'rb'))
  temp = predict_Next_word(model,tokenizer,text)
  return temp

In [83]:
#Converting to the list
li_p = data_p_test.split()

In [84]:
#Converting to the list
li_n = data_n_test.split()

In [85]:
#calculating spacy semantic distance
import spacy
nlp = spacy.load('en')
sum_distance = 0
flag = 0
for i in range(0,len(li_p)-1):
  try:
    temp = Predict_Next_Words_LSTM_P(li_p[i])
    vec1 = nlp(temp)
    vec2 = nlp(li_p[i+1])
    sum_distance+=vec1.similarity(vec2)
    flag+=1
  except:
    continue
print("LSTM Positive Words Accuracy",(sum_distance/flag)*100)

LSTM Positive Words Accuracy 59.48111429185639


In [86]:
import spacy
nlp = spacy.load('en')
sum_distance = 0
flag = 0
for i in range(0,len(li_p)-1):
  try:
    temp = Predict_Next_Words_BILSTM_P(li_p[i])
    vec1 = nlp(temp)
    vec2 = nlp(li_p[i+1])
    sum_distance+=vec1.similarity(vec2)
    flag+=1
  except:
    continue
print("BILSTM Positive Words Accuracy",(sum_distance/flag)*100)

BILSTM Positive Words Accuracy 61.67218205698135


In [87]:
import spacy
nlp = spacy.load('en')
sum_distance = 0
flag = 0
for i in range(0,len(li_n)-1):
  try:
    temp = Predict_Next_Words_LSTM_N(li_n[i])
    vec1 = nlp(temp)
    vec2 = nlp(li_n[i+1])
    sum_distance+=vec1.similarity(vec2)
    flag+=1
  except:
    continue
print("LSTM Negative Words Accuracy",(sum_distance/flag)*100)

LSTM Negative Words Accuracy 51.94569156879323


In [88]:
import spacy
nlp = spacy.load('en')
sum_distance = 0
flag = 0
for i in range(0,len(li_n)-1):
  try:
    temp = Predict_Next_Words_BILSTM_N(li_n[i])
    vec1 = nlp(temp)
    vec2 = nlp(li_n[i+1])
    sum_distance+=vec1.similarity(vec2)
    flag+=1
  except:
    continue
print("BILSTM Negative Words Accuracy",(sum_distance/flag)*100)

BILSTM Negative Words Accuracy 44.73088184947846
